In [2]:
# autoreload is used to reload modules automatically before entering the
# execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
# To import modules from parent directory in Jupyter Notebook
import sys

sys.path.append("..")

In [3]:
import os
import imageio

import numpy as np
import logging
import sys
from scipy.interpolate import RectBivariateSpline

In [4]:
############################# configure logger #############################

# set verbosity
verbosity = 3

level_map = {
    3: logging.DEBUG,
    2: logging.INFO,
    1: logging.WARNING,
    0: logging.ERROR,
}
log_level = level_map[verbosity]
log_handlers = (logging.StreamHandler(sys.stdout),)

logging.basicConfig(
    level=log_level,
    format="[{asctime}] [{levelname:^8s}] [{name:^12s}] <{lineno:^4d}> -- {message:s}",
    style="{",
    datefmt="%H:%M:%S",
    handlers=log_handlers,
)

### Load colored movies (or any other movie...)

In [5]:
training_name = "final_model"
epoch = 100000
movie_id = "05"

In [6]:
data_dir = os.path.join("trainings_validation", training_name, "colored_segmentation")

In [7]:
# change this to select type of annotated movie to load
movie_type = "colored_classes_labels"
filename = f"{training_name}_{epoch}_{movie_id}_{movie_type}.tif"

In [8]:
rgb_movie = np.asarray(imageio.volread(os.path.join(data_dir, filename))).astype(
    np.uint8
)

In [9]:
rgb_movie.shape

(488, 64, 512, 3)

### Upsample movie

In [10]:
# simply repeat each pixel value along x and y axes
scale = 3  # set upscaling factor
upsampled_movie = np.repeat(rgb_movie, scale, axis=1)
upsampled_movie = np.repeat(upsampled_movie, scale, axis=2)

In [29]:
# upsample using linear interpolation
scale = 3  # set upscaling factor

y = np.arange(rgb_movie.shape[1])
x = np.arange(rgb_movie.shape[2])

ynew = np.linspace(0, rgb_movie.shape[1] - 1, rgb_movie.shape[1] * scale)
xnew = np.linspace(0, rgb_movie.shape[2] - 1, rgb_movie.shape[2] * scale)

In [30]:
for frame_id in range(len(rgb_movie)):
    for channel in range(3):
        f = RectBivariateSpline(y, x, rgb_movie[frame_id, :, :, channel])
        upsampled_movie[frame_id, :, :, channel] = f(ynew, xnew)

In [25]:
upsampled_movie.shape

(488, 192, 1536, 3)

### Save movie on disk

In [26]:
# save RGB movie on disk as .mov
# imageio.mimwrite(os.path.join(data_dir, f"{training_name}_{epoch}_{movie_id}_{movie_type}_upsampled_spline.mov"), upsampled_movie, fps=7.5)
imageio.mimwrite(
    os.path.join(
        data_dir,
        f"{training_name}_{epoch}_{movie_id}_{movie_type}_upsampled_spline_smooth.mov",
    ),
    upsampled_movie,
    fps=7.5,
)